# Vertex AI Embedding
Now, let's build a similar classifier to the last one.  But, this time we're going to use data from our graph embedding.

## Setup Variables
First we need to set a few variables.

In [ ]:
# Edit this variable!
REGION = 'us-west1'

# You can leave this as is
shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
PROJECT_ID = shell_output[0]

STORAGE_BUCKET = PROJECT_ID + '-form13'

## Train a Model with Vertex AI AutoML
We'll use the original and engineered features to train an AutoML model.

In [ ]:
from google.cloud import aiplatform
import os

aiplatform.init(project=PROJECT_ID, location=REGION)

dataset = aiplatform.TabularDataset.create(
    display_name="form13-embedding",
    gcs_source=os.path.join("gs://", STORAGE_BUCKET, 'form13', 'embedding.csv'),
)
dataset.wait()

print(f'\tDataset: "{dataset.display_name}"')
print(f'\tname: "{dataset.resource_name}"')

In [ ]:
job = aiplatform.AutoMLTabularTrainingJob(
    display_name='form13-embedding',
    optimization_prediction_type='classification'
)

In [ ]:
model = job.run(
    dataset=dataset,
    target_column='target',
    predefined_split_column_name='split',
    model_display_name='form13-embedding',
    disable_early_stopping=False,
    budget_milli_node_hours=1000,
)

1000 milli node hours, or one node hour, is the minimum budget that Vertex AI allows.  However, Vertex AI isn't respecting that budget currently.  This job will probably run for two and a half hours.  

We're going to move on while that runs.  You can check on the job later in the [Google Cloud Console](https://console.cloud.google.com/) to see the results.  There's a link to the specific job in the output of the cell above.